## Training Deeplabcut for snout and tailbase detection


### Deeplabcut

In [1]:
import deeplabcut

In [2]:
task='tracking' # Enter the name of your experiment Task
experimenter='user' # Enter the name of the experimenter
video=['..\\videos\\video_dlc_single.avi'] # Enter the paths of your videos you want to grab frames from.
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory='..\\dlc_models',copy_videos=True) 

# Modify the config file to have the body parts as follows: snout, leftear, rightear, tailbase

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [7]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file,'automatic','kmeans') 

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: D:\tensorflow\mice_tracking_pipeline\dlc_models\tracking-user-2020-09-21\videos\video_dlc_single.avi ?
yes/noyes


98it [00:00, 979.94it/s]

Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 33.33  seconds.
Extracting and downsampling... 1000  frames from the video.


1000it [00:00, 1033.00it/s]


Kmeans clustering ... (this might take a while)
Frames were successfully extracted.

You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## Label the extracted frames

Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [8]:
%gui wx
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [9]:
#path_config_file = 'C:\\vananh\\DeepLabCut\\vananh\\Tracking-Vananh-2019-08-13\\config.yaml'
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by user.
They are stored in the following folder: D:\tensorflow\mice_tracking_pipeline\dlc_models\tracking-user-2020-09-21\labeled-data\video_dlc_single_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [3]:
deeplabcut.create_training_dataset(path_config_file)

D:\tensorflow\mice_tracking_pipeline\dlc_models\tracking-user-2020-09-21\training-datasets\iteration-0\UnaugmentedDataSet_trackingSep21  already exists!
Currently only ResNet 50, 101 or 152 supported, please change 'resnet' entry in config.yaml!


## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**